# Python standard libraries

In [20]:
import json
import os
import sqlite3

# Third-party libraries

In [21]:
from flask import Flask, redirect, request, url_for
from flask_login import (
    LoginManager,
    current_user,
    login_required,
    login_user,
    logout_user,
)
from oauthlib.oauth2 import WebApplicationClient
import requests

# Internal imports

In [22]:
from db import init_db_command
from user import User

ImportError: cannot import name 'init_db_command' from 'db' (/Users/ehaas/Documents/Python/MyNotebooks/auth/db.py)

# Configuration

In [23]:
GOOGLE_CLIENT_ID = "GOOGLE_CLIENT_ID"
GOOGLE_CLIENT_SECRET = "GOOGLE_CLIENT_SECRET"
# use environmental variables instead to hide these secrets
#GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
#GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = (
    "https://accounts.google.com/.well-known/openid-configuration"
)

# Flask app setup

In [24]:
app = Flask(__name__)
app.secret_key = os.environ.get("SECRET_KEY") or os.urandom(24)

# User session management setup

In [25]:
# https://flask-login.readthedocs.io/en/latest
login_manager = LoginManager()
login_manager.init_app(app)

# Naive database setup

In [26]:
try:
    init_db_command()
except sqlite3.OperationalError:
    # Assume it's already been created
    pass

NameError: name 'init_db_command' is not defined

# OAuth 2 client setup

In [27]:
client = WebApplicationClient(GOOGLE_CLIENT_ID)

# Flask-Login helper to retrieve a user from our db

In [28]:
@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)

# Homepage
This is nothing fancy visually, but you’ll add some neat logic to display something different if a user is logged in. When they’re not logged in, a link will appear that says Google Login.

In [29]:
@app.route("/")
def index():
    if current_user.is_authenticated:
        return (
            "<p>Hello, {}! You're logged in! Email: {}</p>"
            "<div><p>Google Profile Picture:</p>"
            '<img src="{}" alt="Google profile pic"></img></div>'
            '<a class="button" href="/logout">Logout</a>'.format(
                current_user.name, current_user.email, current_user.profile_pic
            )
        )
    else:
        return '<a class="button" href="/login">Google Login</a>'


# Login
Now let’s get to the OAuth 2 flow. The Google Login button from above will redirect to this endpoint. The first step in the flow is to figure out where Google’s OAuth 2 Authorization endpoint is.

Here’s where the lines between what’s defined by OAuth 2 and by OpenID Connect (OIDC) start to blur. As discussed previously, OIDC has a standard endpoint for a provider configuration, which contains a bunch of OAuth 2 and OIDC information. The document with that information is served from a standard endpoint everywhere, .well-known/openid-configuration.

In [30]:
def get_google_provider_cfg():
    return requests.get(GOOGLE_DISCOVERY_URL).json()

The field from the provider configuration document you need is called authorization_endpoint. This will contain the URL you need to use to initiate the OAuth 2 flow with Google from your client application.

In [31]:
@app.route("/login")
def login():
    # Find out what URL to hit for Google login
    google_provider_cfg = get_google_provider_cfg()
    authorization_endpoint = google_provider_cfg["authorization_endpoint"]

    # Use library to construct the request for Google login and provide
    # scopes that let you retrieve user's profile from Google
    request_uri = client.prepare_request_uri(
        authorization_endpoint,
        redirect_uri=request.base_url + "/callback",
        scope=["openid", "email", "profile"],
    )
    return redirect(request_uri)

# Login Callback
Let’s do this one in pieces, since it’s a bit more involved than the previous few endpoints.

The login endpoint on your application is the jumping point for all of Google’s work authenticating the user and asking for consent. Once the user logs in with Google and agrees to share their email and basic profile information with your application, Google generates a unique code that it sends back to your application.

As a reminder, here are the OIDC steps you read about earlier:

1. You register a third-party application as a client to the provider.
1. The client sends a request to the provider’s authorization URL.
1. The provider asks the user to authenticate (prove who they are).
1. The provider asks the user to consent to the client acting on their behalf.
1. The provider sends the client a unique authorization code
1. The client sends the authorization code back to the provider’s token URL
1. The provider sends the client tokens to use with other URLs on behalf of the user

When Google sends back that unique code, it’ll be sending it to this login callback endpoint on your application. So your first step is to define the endpoint and get that code:

In [40]:
@app.route("/login/callback")
def callback():
    # Get authorization code Google sent back to you
    code = request.args.get("code")

    # To start with, you need to figure out what Google’s token endpoint is. You’ll use the provider configuration document again:

    google_provider_cfg = get_google_provider_cfg()
    token_endpoint = google_provider_cfg["token_endpoint"]

    # Prepare and send a request to get tokens! Yay tokens!
    token_url, headers, body = client.prepare_token_request(
        token_endpoint,
        authorization_response=request.url,
        redirect_url=request.base_url,
        code=code
    )
    token_response = requests.post(
        token_url,
        headers=headers,
        data=body,
        auth=(GOOGLE_CLIENT_ID, GOOGLE_CLIENT_SECRET),
    )

    # Parse the tokens!
    client.parse_request_body_response(json.dumps(token_response.json()))

    # Now that you have tokens (yay) let's find and hit the URL
    # from Google that gives you the user's profile information, including their Google profile image and email

    userinfo_endpoint = google_provider_cfg["userinfo_endpoint"]
    uri, headers, body = client.add_token(userinfo_endpoint)
    userinfo_response = requests.get(uri, headers=headers, data=body)

    # parse the response from the userinfo endpoint. 
    # Google uses an optional field, email_verified, to confirm that not only has the user created an account, but they’ve verified the email address to complete the account creation. It’s generally safe to conditionally check for this verification, as it’s another layer of security that Google offers.
    # That being said, you’ll check for that, and if Google says the user is verified, then you’ll parse their information. The 4 pieces of basic profile information you’ll use are:

    # 1. sub: the subject, a unique identifier for the user in Google
    # 2. email: the user’s Google email address
    # 3. picture: the user’s public profile picture in Google
    # 4. given_name: the user’s first and last name in Google

    if userinfo_response.json().get("email_verified"):
        unique_id = userinfo_response.json()["sub"]
        users_email = userinfo_response.json()["email"]
        picture = userinfo_response.json()["picture"]
        users_name = userinfo_response.json()["given_name"]
    else:
        return "User email not available or not verified by Google.", 400

    # The final steps in this callback are:

    # 1. Create a user in your database with the information you just got from Google
    # 2. Begin a user session by logging that user in
    # 3. Send user back to the homepage (where you’ll now display their public profile information)

    # Create a user in your db with the information provided
    # by Google
    user = User(
        id_=unique_id, name=users_name, email=users_email, profile_pic=picture
    )

    # Doesn't exist? Add it to the database.
    if not User.get(unique_id):
        User.create(unique_id, users_name, users_email, picture)

    # Begin user session by logging the user in
    login_user(user)

    # Send user back to homepage
    return redirect(url_for("index"))

AssertionError: View function mapping is overwriting an existing endpoint function: callback

# Logout
The logout endpoint is much less code than the last few endpoints. You just call a logout function and redirect back to the homepage. Done and done. Here it i

In [38]:
@app.route("/logout")
@login_required
def logout():
    logout_user()
    return redirect(url_for("index"))